# Script para etiquetar las imagenes de entrenamiento
## Entorno de ejecución: Colab
## Autor: Juan M. Mendoza

In [2]:
# Descarga el dataset
!gdown "https://drive.google.com/uc?export=download&id=12KR_l0TgLhwc3J25GFUkd9S73OEtphG_"
!gdown "https://drive.google.com/uc?export=download&id=12ELJm48gudZApmxozqqeg4JL5HInDauU"

Downloading...
From: https://drive.google.com/uc?export=download&id=12KR_l0TgLhwc3J25GFUkd9S73OEtphG_
To: /content/images_ts.rar
147MB [00:01, 110MB/s] 
Downloading...
From: https://drive.google.com/uc?export=download&id=12ELJm48gudZApmxozqqeg4JL5HInDauU
To: /content/images_tr.rar
342MB [00:05, 63.4MB/s]


In [3]:
# instala el compresor
!apt-get install rar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 295 kB of archives.
After this operation, 799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 rar amd64 2:5.5.0-1 [295 kB]
Fetched 295 kB in 1s (461 kB/s)
Selecting previously unselected package rar.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1_amd64.deb ...
Unpacking rar (2:5.5.0-1) ...
Setting up rar (2:5.5.0-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [5]:
# extrae los archivos
! rar x 'images_tr.rar'


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help


Extracting from images_tr.rar

Creating    images_tr                                                 OK
Creating    images_tr/56                                              OK
Extracting  images_tr/56/spleen_56.ni_z026.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z014.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z008.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z031.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z030.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z032.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z001.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z019.png               

In [10]:
# importa las librerias que se usaran en el etiquetado
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [18]:
import os, sys
import glob

dirs = os.listdir('images_tr')
for file in dirs:
  #obtiene los paths para entrenamiento
  paths_tr = glob.glob("images_tr/"+file+"/*.png")
  os.makedirs("images_label_tr/"+file)
  for path in paths_tr:
    # lee la imagen y la pasa a escala de grises
    img = cv.imread(path)
    print(path)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    # Filtrado de la imagen 
    # eliminacion de ruido
    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
    # area de fondo
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    # area de interes
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # encontrando la region desconocida
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)
    # https://stackoverflow.com/questions/22240746/recognize-open-and-closed-shapes-opencv
    contours, hierarchy = cv.findContours(unknown, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]

    for i, c in enumerate(contours):
        if hierarchy[i][2] < 0 and hierarchy[i][3] < 0:
            cv.drawContours(img, contours, i, (0, 0, 255), 2)
        else:
            cv.drawContours(img, contours, i, (0, 255, 0), 2)
    # ploteo de region desconocida
    #plt.axis("off")
    #plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    #plt.show()
    cv.imwrite( "images_label_tr/"+file+"/label_"+path.split('/')[2], img);

images_tr/56/spleen_56.ni_z026.png
images_tr/56/spleen_56.ni_z014.png
images_tr/56/spleen_56.ni_z008.png
images_tr/56/spleen_56.ni_z031.png
images_tr/56/spleen_56.ni_z030.png
images_tr/56/spleen_56.ni_z032.png
images_tr/56/spleen_56.ni_z001.png
images_tr/56/spleen_56.ni_z019.png
images_tr/56/spleen_56.ni_z028.png
images_tr/56/spleen_56.ni_z009.png
images_tr/56/spleen_56.ni_z015.png
images_tr/56/spleen_56.ni_z017.png
images_tr/56/spleen_56.ni_z029.png
images_tr/56/spleen_56.ni_z011.png
images_tr/56/spleen_56.ni_z025.png
images_tr/56/spleen_56.ni_z004.png
images_tr/56/spleen_56.ni_z013.png
images_tr/56/spleen_56.ni_z016.png
images_tr/56/spleen_56.ni_z010.png
images_tr/56/spleen_56.ni_z003.png
images_tr/56/spleen_56.ni_z018.png
images_tr/56/spleen_56.ni_z021.png
images_tr/56/spleen_56.ni_z012.png
images_tr/56/spleen_56.ni_z002.png
images_tr/56/spleen_56.ni_z022.png
images_tr/56/spleen_56.ni_z024.png
images_tr/56/spleen_56.ni_z027.png
images_tr/56/spleen_56.ni_z020.png
images_tr/56/spleen_

In [20]:
# comprime los archivos etiquetados
!rar a "images_label_tr" "images_label_tr"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive images_label_tr.rar

Adding    images_label_tr/56/label_spleen_56.ni_z031.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z028.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z033.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z008.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z016.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z001.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z018.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z003.png                   0%  OK 
Adding    images_label_tr/56/label_spleen_56.ni_z002.png                 